# 10-3 MNIST-CNN woth Visdom

In [8]:
import torch
import torchvision.datasets as dsets
import torch.nn as nn
import torchvision.transforms as transforms

import torch.nn.init

### import visdom

In [9]:
import visdom
vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

### define loss_tracker

In [10]:
def loss_tracker(loss_plot, loss_value, num):  # update할 plot의 이름, loss값이 무엇인지, index(loss값이 들어갈 자리)
    # num&loss_value, are Tensor
    vis.line(X=num,
             Y=loss_value,
             win=loss_plot,
             update='append'
            )

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)  #random value 고정 코드
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [12]:
#parameters 
learning_rate = 0.001
training_epochs = 15
batch_size = 32

In [13]:
#MNIST dataset

mnist_train = dsets.MNIST(root='MNIST_data/',
                        train = True,
                        transform = transforms.ToTensor(),  # -> input data를 어떻게 변환할지
                                                            # 그냥 받아오면 Tensor value가 아니라서 mnist_data를 Tensor value로 전환
                        download = True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                        train = False,
                        transform = transforms.ToTensor(),  # tensor로 변환
                        download = True)


In [14]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True, # dataset을 섞은 상태로 던져준다.
                                          drop_last=True)

In [15]:
#model  -> cnn 모듈 만들기

class CNN(nn.Module):
    
    def __init__(self):  # 초기함수 -> 이거 빼먹으면 학습 안됨.
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(   # nn.Sequential은 그냥 치고 내가 원하는걸 만들어서 넣는 방식으로 사용
            nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1), # layer 한 개 쌓임
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc = nn.Linear(7*7*64, 10, bias = True) 
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        
        return out

In [19]:
model = CNN().to(device)

# model 테스트
value = (torch.Tensor(1,1,28,28)).to(device)
print( (model(value)).shape  )

torch.Size([1, 10])


In [20]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

### make plot
10-2랑 다른 부분

In [21]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

### train with loss_tracker

In [23]:
#training
total_batch = len(data_loader) # batch_size가 몇인지를 알 수 있음 (전체의 batch가 몇인지)

for epoch in range(training_epochs):
    avg_cost = 0 #loss를 담을 것
    
    for X, Y in data_loader: # X = input image, Y = label
        # CUDA 연산을 진행하려면 torch.Tensor가 아니라 torch.cuda.Tensor가 되어있어야함
        # 없으면 아예 학습이 안된다.
        # input data, label data를 device에 맞춰서 설정해주는 작업
        X = X.to(device)
        Y = Y.to(device)   
        
        optimizer.zero_grad()
        hypothesis = model(X) # model에 input값 넣은 출력값
        
        # model에 input 값을 집어넣고 나온 cost는 cross_entropy로 계산
        cost = criterion(hypothesis, Y) # 가설과 실제 label 사이 차이 계산
        cost.backward()
        optimizer.step() # cost.backward()값을 토대로 model을 학습시키는 과정
        
        avg_cost += cost / total_batch
        
    
    print('[Epoch:{}] cost = {}'.format(epoch+1,avg_cost))
    ##추가된 부분
    ### 여기서 loss_plt는 업데이트할 그래프 이름
    ### 업데이트하는 X index는 epoch
    ### 한 epoch이 끝날 때마다 loss 값의 평균(avg_cost)을 넣는다는 뜻
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
    ##
print('Learning Finished!')

[Epoch:1] cost = 0.05169275030493736
[Epoch:2] cost = 0.037330009043216705
[Epoch:3] cost = 0.0285178292542696
[Epoch:4] cost = 0.021362457424402237
[Epoch:5] cost = 0.017661433666944504
[Epoch:6] cost = 0.013687617145478725
[Epoch:7] cost = 0.010581240057945251
[Epoch:8] cost = 0.008344694040715694
[Epoch:9] cost = 0.008169867098331451
[Epoch:10] cost = 0.006502591073513031
[Epoch:11] cost = 0.006269106641411781
[Epoch:12] cost = 0.005767275579273701
[Epoch:13] cost = 0.005023234058171511
[Epoch:14] cost = 0.0037444753106683493
[Epoch:15] cost = 0.003813603427261114
Learning Finished!
